In [1]:
import os
import sys

app_path = os.getcwd().rsplit(os.sep, 2)[0]

"""Add app path to sys.path for importing parent directory modules"""
if app_path not in sys.path:
    sys.path.insert(0, app_path)

# Note: Do NOT delete this cell

In [22]:
import pandas as pd

from utils.paths import data_path
from utils.helpers import preprocess,get_hashtags

In [23]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import nltk

nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, recall_score, mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.svm import SVR
from typing import Optional

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
path = data_path('train.csv')
df   = pd.read_csv(path)

df.head()

,tweet_id,tweet,sentiment
0,1701,#sxswnui #sxsw #apple defining language of tou...,1
1,1851,Learning ab Google doodles! All doodles should...,1
2,2689,one of the most in-your-face ex. of stealing t...,2
3,4525,This iPhone #SXSW app would b pretty awesome i...,0
4,3604,Line outside the Apple store in Austin waiting...,1


In [10]:
#import re

#def get_hashtags(text, handle_space=True):

 #   if handle_space:
  #      space = r"#\s+"
   #     text = re.sub(space, "#", str(text).lower()).strip()
    
    #hash_tags = r"#\S+"
    
   # return " ".join(re.findall(hash_tags, text))

In [25]:
df['hastags']=df.tweet.apply(get_hashtags, 1)

In [26]:
df.head()

,tweet_id,tweet,sentiment,hastags
0,1701,#sxswnui #sxsw #apple defining language of tou...,1,#sxswnui #sxsw #apple
1,1851,Learning ab Google doodles! All doodles should...,1,#googledoodle #sxsw
2,2689,one of the most in-your-face ex. of stealing t...,2,"#sxsw,"
3,4525,This iPhone #SXSW app would b pretty awesome i...,0,#sxsw #fuckit #illmakeitwork
4,3604,Line outside the Apple store in Austin waiting...,1,#sxsw


In [27]:
def preprocess(text, stem=False):
    # Remove links, users and special characters
    TEXT_CLEANING_RE = r"@mention|https?:\S+|http?:\S|[^A-Za-z0-9]+"
    stop_words       = stopwords.words("english")
    stemmer          = SnowballStemmer("english")

    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip().replace("rt", "")
    tokens = []

    for token in text.split():

        if token not in stop_words:

            if stem:
                tokens.append(stemmer.stem(token))

            else:
                tokens.append(token)

    return " ".join(tokens)

In [28]:
df.tweet.apply(preprocess, 1)
df.head()

,tweet_id,tweet,sentiment,hastags
0,1701,#sxswnui #sxsw #apple defining language of tou...,1,#sxswnui #sxsw #apple
1,1851,Learning ab Google doodles! All doodles should...,1,#googledoodle #sxsw
2,2689,one of the most in-your-face ex. of stealing t...,2,"#sxsw,"
3,4525,This iPhone #SXSW app would b pretty awesome i...,0,#sxsw #fuckit #illmakeitwork
4,3604,Line outside the Apple store in Austin waiting...,1,#sxsw


In [30]:
df['tweet'][2]

'one of the most in-your-face ex. of stealing the show in yrs RT @mention &quot;At #SXSW, Apple schools the mkt experts&quot;  {link}'